In [41]:
import os
from pathlib import Path
import random

import puppets_data
import ConnectionGraphX
import GraphX

import numpy as np
import pandas as pd
import scipy
import networkx as nx

import ConnectionNetworkX as cnx

from pyLDLE2 import datasets
from pyLDLE2 import buml_
from pyLDLE2 import util_

from proxi.algorithms.knng import get_knn_graph
from proxi.utils.misc import save_graph, save_weighted_graph
from proxi.utils.process import *
from proxi.utils.distance import abs_correlation

Initializing and pre-processing the puppets data.

In [42]:
p = "/Users/sawyerjack/IdeaProjects/project_data/PuppetsData"
save_dir_root = "/Users/sawyerjack/IdeaProjects/connection-graph-wasserstein/data"

## Just for time and sanity, using n = 100 while I test things.
N_IMAGES = 2500

In [43]:
Y, labelsMat, _ = puppets_data.puppets_data(p)

X.shape =  (8100, 100)


Select `N_IMAGES` uniformly at random from the dataset.

In [44]:
X = np.array(Y.copy())
totalImages = X.shape[0]
sampleIndices = random.sample(range(totalImages), N_IMAGES)
X = [ X[i] for i in sampleIndices ]

X = np.array(X)

Initialize a model and use Local PCA at each vertex neighborhood.

In [46]:
N_NEAREST_NEIGHBORS = 28

buml_obj = buml_.BUML(local_opts={'algo':'LPCA', 'k': N_NEAREST_NEIGHBORS},
                      vis_opts={'c': labelsMat[:,0], 'save_dir': save_dir_root},
                      verbose=True, debug=True, exit_at='local_views')

buml_obj.fit(X=X)


local_opts['k_nn0'] = 700 is created.
Options provided:
local_opts:
{
    "Atilde_method": "LDLE_1",
    "N": 100,
    "U_method": "k_nn",
    "algo": "LPCA",
    "alpha": 1,
    "debug": true,
    "delta": 0.9,
    "gl_type": "unnorm",
    "k": 28,
    "k_nn": 49,
    "k_nn0": 700,
    "k_tune": 7,
    "lambda1_decay": 0.75,
    "lambda1_init": 8,
    "lambda1_min": 0.001,
    "max_iter": 300,
    "max_sparsity": 0.9,
    "metric": "euclidean",
    "n_proc": 6,
    "p": 0.99,
    "power": 5,
    "pp_n_thresh": 32,
    "radius": 0.5,
    "reg": 0.0,
    "scale_by": "gamma",
    "tau": 50,
    "to_postprocess": true,
    "verbose": true
}
intermed_opts:
{
    "algo": "best",
    "debug": true,
    "eta_max": 25,
    "eta_min": 5,
    "len_S_thresh": 256,
    "local_algo": "LPCA",
    "metric": "euclidean",
    "n_proc": 6,
    "n_times": 4,
    "verbose": true
}
global_opts:
{
    "add_dim": false,
    "align_transform": "rigid",
    "align_w_parent_only": true,
    "alpha": 0.3,
    "b

Read in the intrinsic dimension of the CNX and then initialize it by symmetrizing the kNN graph.

In [47]:
INTRINSIC_DIMENSION = 2

# U is a directed boolean adjacency matrix, so we symmetrize by adding U.T.
# This weights directed and double edges equally.
puppetGraphAdjacency = buml_obj.LocalViews.U + buml_obj.LocalViews.U.T

puppetGraph = cnx.ConnectionNetworkX(puppetGraphAdjacency, INTRINSIC_DIMENSION)


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/scipy/sparse/_index.py:103: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


Interpolate the local PCA views by registering the overlap of adjacent nodes' neighborhoods, and then using orthogonal procrustes to align the views. This becomes the connection at the corresponding edge.

In [48]:
for i in range(N_IMAGES):
    n_i = nx.neighbors(puppetGraph, i)
    for j in [j for j in n_i if j > i]:

        n_ij = buml_obj.LocalViews.U[i,:].multiply(buml_obj.LocalViews.U[j,:]).nonzero()[1]

        if len(n_ij) >= 2:
            X_Uij_i = buml_obj.LocalViews.local_param_post.eval_({'view_index': i, 'data_mask': n_ij})
            X_Uij_j = buml_obj.LocalViews.local_param_post.eval_({'view_index': j, 'data_mask': n_ij})

            Tij, _ = util_.custom_procrustes(X_Uij_i, X_Uij_j)
        else:
            Tij = np.toa

        puppetGraph.updateEdgeSignature((i,j), Tij)


94 2
94 3
95 2
95 3
374 4
374 5
375 4
375 5
556 6
556 7
557 6
557 7
634 8
634 9
635 8
635 9
660 10
660 11
661 10
661 11
722 12
722 13
723 12
723 13
724 14
724 15
725 14
725 15
776 16
776 17
777 16
777 17
844 18
844 19
845 18
845 19
1000 20
1000 21
1001 20
1001 21
1220 22
1220 23
1221 22
1221 23
1518 24
1518 25
1519 24
1519 25
1850 26
1850 27
1851 26
1851 27
2144 28
2144 29
2145 28
2145 29
2468 30
2468 31
2469 30
2469 31
2706 32
2706 33
2707 32
2707 33
2842 34
2842 35
2843 34
2843 35
3242 36
3242 37
3243 36
3243 37
3244 38
3244 39
3245 38
3245 39
3276 40
3276 41
3277 40
3277 41
3430 42
3430 43
3431 42
3431 43
3464 44
3464 45
3465 44
3465 45
4188 46
4188 47
4189 46
4189 47
4254 48
4254 49
4255 48
4255 49
4302 50
4302 51
4303 50
4303 51
4502 52
4502 53
4503 52
4503 53
4576 54
4576 55
4577 54
4577 55
4974 56
4974 57
4975 56
4975 57
216 60
216 61
217 60
217 61
402 62
402 63
403 62
403 63
420 64
420 65
421 64
421 65
542 66
542 67
543 66
543 67
768 68
768 69
769 68
769 69
816 70
816 71
817 70

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pyLDLE2/util_.py:221: RuntimeWarning: invalid value encountered in divide
  X0 /= normX
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pyLDLE2/util_.py:222: RuntimeWarning: invalid value encountered in divide
  Y0 /= normY


LinAlgError: SVD did not converge

In [49]:
puppetGraph.connectionLaplacianMatrix.shape

(5000, 5000)

In [54]:
scipy.sparse.linalg.eigsh(puppetGraph.connectionLaplacianMatrix, k=20, which="SM", return_eigenvectors=False)

array([4.18483411, 4.06878222, 4.00767116, 3.87354483, 3.82885997,
       3.75505086, 3.5745064 , 3.51348364, 3.43939002, 3.37575573,
       3.3530883 , 3.16693467, 3.1153024 , 3.01655908, 2.86975347,
       2.73143024, 2.53019541, 2.38996075, 2.35161071, 1.99786154])